# How to download crypto data

CSV files in the _data_ folder contains TOHLCV (time, open, high, low, close, volume) crypto data with a resolution equals to 5 minutes. The considered pairs are: BTC/USD, ETH/USD, SOL/USD, BNB/USD, USDT/USD, LTC/USD, DOGE/USD. The considered period is 2021/12/22 - 2022/01/02. Data have been retrieved using the open source Python package named _ccxt_ (https://docs.ccxt.com/en/latest/manual.html). The considered exchange is FTX (https://ftx.com). A jupyter notebook showing the downloading process is provided.

In [65]:
import pandas as pd
import time

import ccxt
import calendar
import datetime 

from datetime import date, timedelta

In [66]:
sdate = date(2021,12,25)   # start date -> included
edate = date(2022,1,3)   # end date - excluded

pair = 'DOGE/USD'

In [67]:
list_dates = pd.date_range(sdate,edate-timedelta(days=1),freq='d').strftime("%Y-%m-%d").tolist()

In [68]:
ftx = ccxt.ftx()
final_df = pd.DataFrame({'Time':[], 'Open':[], 'High':[], 'Low':[], 'Close':[], 'Volume':[]})

for date in list_dates:
    since = date
    since = time.mktime(datetime.datetime.strptime(since, "%Y-%m-%d").timetuple()) * 1000
    ohlcv = ftx.fetch_ohlcv(symbol=pair, timeframe='5m', since=since)
    start_dt = datetime.datetime.fromtimestamp(ohlcv[0][0]/1000)
    end_dt = datetime.datetime.fromtimestamp(ohlcv[-1][0]/1000)

    df = pd.DataFrame(ohlcv, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Time'] = [datetime.datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
    df = df.iloc[:12*24, :]
    
    final_df = final_df.append(df, ignore_index=True)
    
final_df.set_index('Time', inplace=True)

In [69]:
pair = pair.replace('/', '-')
final_df.to_csv(f'data_{pair}.csv')

In [70]:
df = pd.read_csv(f'./data_{pair}.csv')

In [71]:
df.set_index('Time', inplace=True)

In [72]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2021-12-25 00:00:00,0.187195,0.187208,0.185744,0.186852,90314.800492
2021-12-25 00:05:00,0.186867,0.187027,0.186454,0.186544,55508.301788
2021-12-25 00:10:00,0.186533,0.186559,0.184204,0.185183,199235.934907
2021-12-25 00:15:00,0.185183,0.185996,0.184837,0.185996,81289.208735
2021-12-25 00:20:00,0.185999,0.186759,0.185780,0.186446,121510.656159
...,...,...,...,...,...
2022-01-02 23:35:00,0.174313,0.174521,0.174299,0.174521,11705.199171
2022-01-02 23:40:00,0.174521,0.174650,0.174497,0.174649,14119.417816
2022-01-02 23:45:00,0.174649,0.174656,0.174147,0.174152,22410.825600
